#### Tropical mean P constructed by events of aggregated convection

In [ ]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import dask.array as da
from scipy.ndimage import gaussian_filter

from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
import psutil
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib import patches
from matplotlib import cm
from matplotlib.colors import ListedColormap, Normalize

from metpy import calc as mpcalc
from metpy.units import units
from scipy.stats import ttest_ind

In [ ]:
ISCCP_dir = '/data/willytsai/ISCCP/hgg/'
TRMM_dir = '/data2/willytsai/TRMM_3hr/TRMM/'
MERRA2_dir = '/data2/willytsai/MERRA2/'
MERRA2_3d_dir = '/data/willytsai/'
ERA5_dir = '/data/willytsai/ERA5/'
Gridsat_dir = '/data/willytsai/gridsat_BT/remap_0.25deg/'
fig_dir = '/data/willytsai/ISCCP/fig/'

In [ ]:
 os.chdir(TRMM_dir+'2014_2018')

In [ ]:
%%time
date = []; date_begin = datetime(2014,1,1)
for t in range(14608):
    tmp = date_begin + t*timedelta(hours=3)
    date.append(tmp)
                
# # TRMM3B42 dataset 2014, 3hrly separate
prec_3hr = np.nan*np.zeros((len(date),241,1440))

In [ ]:
%%time
files = sorted(glob('3B42.*'))
for n,file in enumerate(files):
    try:
        data = xr.open_dataset(file).precipitation        
        prec_3hr[n,:,:] = data.values.swapaxes(0,1)
    except:
        prec_3hr[n,:,:] = np.nan       
lon = data.nlon; lat = data.nlat
prec_xr = xr.DataArray(prec_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del prec_3hr

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# load data
os.chdir('/data/willytsai/ISCCP/')

# variable namelist
var_name = np.loadtxt('var_name_2014_2018_5deg_4ds.dat',dtype='U16')
# event time
var_time = np.loadtxt('time_event_2014_2018_5deg_4ds.dat',dtype='U13')
time_event = []
for t in var_time:
    tmp = datetime.strptime(t,'%Y-%m-%d-%H')
    time_event.append(tmp)

var_dataset = (np.loadtxt('var_event_2014_2018_5deg_4ds.dat'))

In [ ]:
lat_event = var_dataset[:,0]
lon_event = var_dataset[:,1]

In [ ]:
prec_all = np.load('prec_all_5deg_4ds.mat.npy') #
SCAI_all = np.load('SCAIbt_all_5deg_4ds.mat.npy')

In [ ]:
%%time
date_frame = np.asarray(date)

lat_bin = np.linspace(-7.5,7.5,7)
lon_bin = np.linspace(-177.5,177.5,143)

precsum_uncov = np.zeros((len(lon_bin),len(lat_bin)))
precsum_uni = np.copy(precsum_uncov)
precsum_overlap = np.copy(precsum_uncov)

for i,lon_cen in enumerate(lon_bin):
    for j,lat_cen in enumerate(lat_bin):
        
        prec_record = np.zeros(len(date_frame))
        prec_mn = prec_xr.sel(lat=slice(lat_cen-2.5,lat_cen+2.5)
                                         ,lon=slice(lon_cen-2.5,lon_cen+2.5)).mean(axis=(1,2)) # time-series
        idx = np.where(24*prec_mn >=5)[0] # larger than 5mm/day
        for n in idx:
            if n >= 16 and (n + 17) < len(prec_record):
                if prec_mn[n] == np.max(prec_mn[n-16:n+17]):
                    prec_record[n-16:n+17] += 1 # 4-day window
        
        idx_unc = np.where(prec_record==0)[0] # uncovered time
        idx_uni = np.where(prec_record==1)[0] # unitary evnets
        idx_over = np.where(prec_record>1)[0] # overlapped time
        
        precmn_uncov[i,j] = prec_mn[idx_unc].mean()
        precmn_uni[i,j] = prec_mn[idx_uni].mean()
        precsum_overlap[i,j] = prec_mn[idx_over].sum()/prec_mn.sum()

In [ ]:
%%time 
lat_bin = np.linspace(-7.5,7.5,7)
lon_bin = np.linspace(-177.5,177.5,143)

precmn_clim = np.zeros((143,7))
precmn_tot = np.copy(precmn_clim)
precmn_org = np.copy(precmn_clim)
precmn_scat = np.copy(precmn_clim)
for i,lon_cen in enumerate(lon_bin):
    for j,lat_cen in enumerate(lat_bin):
    
        idx_geo = np.where(np.logical_and(lat_event==lat_cen, lon_event==lon_cen))[0]
        idx_SCAI1 = np.where(SCAI_all[:,16] < 3)[0]
        idx_SCAI2 = np.where(SCAI_all[:,16] > 6)[0]        
        idx_org = np.intersect1d(idx_geo,idx_SCAI1)
        idx_scat = np.intersect1d(idx_geo,idx_SCAI2)
        
        precmn_clim[i,j] = prec_xr.sel(lat=slice(lat_cen-2.5,lat_cen+2.5)
                                    ,lon=slice(lon_cen-2.5,lon_cen+2.5)).mean() # climatology P in a single 5x5 box
        
        precmn_tot[i,j] = np.nanmean(prec_all[idx_geo,:])
        precmn_org[i,j] = np.nanmean(prec_all[idx_org,:]) # 
        precmn_scat[i,j] = np.nanmean(prec_all[idx_scat,:])

In [ ]:
plt.hist(SCAI_all[:,16],bins=20);

In [ ]:
precmn_clim_re = np.zeros((145,7))
precmn_clim_re[:143,:] = precmn_clim
precmn_clim_re[143,:] = (precmn_clim[0,:]+precmn_clim[-1,:])/2
precmn_clim_re[144,:] = (precmn_clim[0,:]+precmn_clim[-1,:])/2    
precmn_clim_re = np.vstack([precmn_clim_re.T,np.zeros((precmn_clim_re.shape[0])).T]).T

lat_bin = np.linspace(-8.75,8.75,8)
lon_bin = np.linspace(-178.75,181.25,145)

In [ ]:
precmn_org_re = np.zeros((145,7))
precmn_org_re[:143,:] = precmn_org
precmn_org_re[143,:] = (precmn_org[0,:]+precmn_org[-1,:])/2
precmn_org_re[144,:] = (precmn_org[0,:]+precmn_org[-1,:])/2    
precmn_org_re = np.vstack([precmn_org_re.T,np.zeros((precmn_org_re.shape[0])).T]).T

In [ ]:
precmn_scat_re = np.zeros((145,7))
precmn_scat_re[:143,:] = precmn_scat
precmn_scat_re[143,:] = (precmn_scat[0,:]+precmn_scat[-1,:])/2
precmn_scat_re[144,:] = (precmn_scat[0,:]+precmn_scat[-1,:])/2    
precmn_scat_re = np.vstack([precmn_scat_re.T,np.zeros((precmn_scat_re.shape[0])).T]).T

In [ ]:
precmn_tot_re = np.zeros((145,7))
precmn_tot_re[:143,:] = precmn_tot
precmn_tot_re[143,:] = (precmn_tot[0,:]+precmn_tot[-1,:])/2
precmn_tot_re[144,:] = (precmn_tot[0,:]+precmn_tot[-1,:])/2    
precmn_tot_re = np.vstack([precmn_tot_re.T,np.zeros((precmn_tot_re.shape[0])).T]).T

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,8.5)
        ,subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

#prec_con,lon = add_cyclic_point(prec_clim, coord=prec.lon)
cf = ax.pcolormesh(lon_bin,lat_bin,24*precmn_tot_re.T,transform=ccrs.PlateCarree(),cmap='jet')
ax.set_title('TRMM precipitation mean, 5-yr climatology',fontsize=14)

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-180,-150,-120,-90,-60,-30,-0,30,60,90,120,150,180])
ax.set_xticklabels(['0','30E','60E','90E','120E','150E','180','150W','120W','90W','60W','30W','0'])
ax.set_ylim([-10,10]);ax.grid(linestyle=':')
ax.set_xlim([-129,89])
cbar = plt.colorbar(cf,ax=ax,shrink=0.3,pad=0.02)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,10)
        ,subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

#prec_con,lon = add_cyclic_point(prec_clim, coord=prec.lon)
cf = ax.pcolormesh(lon_bin,lat_bin,24*precmn_org_re.T,transform=ccrs.PlateCarree(),cmap='jet')
ax.set_title('TRMM precipitation mean, Aggregated events, SCAI < 4',fontsize=14)

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-180,-150,-120,-90,-60,-30,-0,30,60,90,120,150,180])
ax.set_xticklabels(['0','30E','60E','90E','120E','150E','180','150W','120W','90W','60W','30W','0'])
ax.set_ylim([-10,10]);ax.grid(linestyle=':')
ax.set_xlim([-129,89])
cbar = plt.colorbar(cf,ax=ax,shrink=0.3,pad=0.02)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,8.5)
        ,subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

#prec_con,lon = add_cyclic_point(prec_clim, coord=prec.lon)
cf = ax.pcolormesh(lon_bin,lat_bin,24*precmn_scat_re.T,transform=ccrs.PlateCarree(),cmap='jet')
ax.set_title('TRMM precipitation mean, scattered events, SCAI > 8',fontsize=14)

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-180,-150,-120,-90,-60,-30,-0,30,60,90,120,150,180])
ax.set_xticklabels(['0','30E','60E','90E','120E','150E','180','150W','120W','90W','60W','30W','0'])
ax.set_ylim([-10,10]);ax.grid(linestyle=':')
ax.set_xlim([-129,89])
cbar = plt.colorbar(cf,ax=ax,shrink=0.3,pad=0.02)

In [ ]:
lon_bin.shape

In [ ]:
precmn_tot_re.shape

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,10)
        ,subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

cf = ax.pcolormesh(lon_bin,lat_bin,24*(precmn_org_re-precmn_tot_re).T,transform=ccrs.PlateCarree()
                   ,cmap='BrBG',vmin=-2,vmax=2)
ax.set_title('TRMM precipitation mean, Aggregated events, SCAI < 3',fontsize=14)
#prec_con,lon = add_cyclic_point(precmn_tot_re, coord=lon_bin)
ax.contour(lon_bin,lat_bin,24*precmn_tot_re.T,transform=ccrs.PlateCarree(),
             levels=[5,7,10,15],colors=['k'],linewidths=1)

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-180,-150,-120,-90,-60,-30,-0,30,60,90,120,150,180])
ax.set_xticklabels(['0','30E','60E','90E','120E','150E','180','150W','120W','90W','60W','30W','0'])
ax.set_ylim([-10,10]);ax.grid(linestyle=':')
ax.set_xlim([-129,89])
cbar = plt.colorbar(cf,ax=ax,shrink=0.3,pad=0.05,orientation='horizontal')